# Final Resource

final resource as a RMarkdown webpage

- The purpose of your topic is clearly captured in your resource
- Quantitative questions are clearly and concisely explained with thoughtful text and compelling visuals
- A nuanced understanding of the important features of the dataset and topic is demonstrated.
- High-level insights (important descriptive information, major trends, notable outliers, etc.) should be prominent in your resource.
- Methods and results of statistical approaches are clear
- Your analysis should be easily reproducible using information in your GitHub repository
- Remember to tailor your resource to a specific target audience. The amount of framing you need to do for a scientific versus general audience is quite different

https://www.kaggle.com/pvkc8888/dota-2-pro-circuit-1718

In [135]:
# Import packages
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from sklearn import metrics

In [152]:
dpc = pd.read_json("data/DPC.json").transpose()

In [158]:
dpc.drop(["all_word_counts", "barracks_status_dire", "barracks_status_radiant", "chat", "cluster", "comeback", 
          "cosmetics", "dire_score", "series_type", "skill", "stomp", "teamfights", "throw", "tower_status_dire", 
          "tower_status_radiant", "loss", "leagueid", "duration", "engine", "first_blood_time", "radiant_xp_adv", 
          "replay_url", "radiant_gold_adv", "players", "positive_votes", "radiant_score", "negative_votes", 
          "objectives", "dire_team_id", "radiant_team_id", "human_players", "league", "lobby_type", 
          "draft_timings", "my_word_counts", "region", "replay_salt", "series_id"], axis=1, inplace=True)
dpc.drop([1], inplace=True)

In [319]:
# patches = dpc["patch"].unique()
# for patch in patches:
#     print("in patch " + str(patch) + ": " + str(dpc.loc[dpc["patch"] == patch].shape[0]) + " matches")
#     print("    first instance of patch: " + str(dpc.loc[dpc["patch"] == patch].head(1).index[0]))
#     print("     last instance of patch: " + str(dpc.loc[dpc["patch"] == patch].tail(1).index[0]))
# print(dpc.columns)
# print(dpc.at[3625490002, 'picks_bans'])

In [315]:
# (7.06, 7.07, 7.08, 7.09, 7.10, 7.11, 7.12, 7.13, 7.14, 7.15, 7.16, 7.16)
# major patches
# patch 7.07 to 7.09 (patch 26 to 28)
# patch 7.10 to 7.13 (patch 29 to 32)
# patch 7.14 to 7.16 (patch 33 to 36)

# for data with picks and bans
# for hero data https://github.com/kronusme/dota2-api/blob/master/data/heroes.json
dpc_df = pd.DataFrame(columns=["pick_1", "pick_2", "pick_3", "pick_4", "pick_5",
                                 "ban_1", "ban_2", "ban_3", "ban_4", "ban_5", "ban_6",
                                 "win"])

patch_range_low = 26
patch_range_high = 28

for i in dpc.index:
    patch = dpc.at[i, 'patch']
    game_mode = dpc.at[i, 'game_mode']
    if patch >= patch_range_low and patch <= patch_range_high and game_mode == 2: 
        length = len(dpc.at[i, 'picks_bans'])
        if  length == 22:
            ban1 = dpc.at[i, 'picks_bans'][0]["hero_id"]
            ban6 = dpc.at[i, 'picks_bans'][1]["hero_id"]
            ban2 = dpc.at[i, 'picks_bans'][2]["hero_id"]
            ban7 = dpc.at[i, 'picks_bans'][3]["hero_id"]
            ban3 = dpc.at[i, 'picks_bans'][4]["hero_id"]
            ban8 = dpc.at[i, 'picks_bans'][5]["hero_id"]

            pick1 = dpc.at[i, 'picks_bans'][6]["hero_id"]
            pick6 = dpc.at[i, 'picks_bans'][7]["hero_id"]
            pick7 = dpc.at[i, 'picks_bans'][8]["hero_id"]
            pick2 = dpc.at[i, 'picks_bans'][9]["hero_id"]

            ban9 = dpc.at[i, 'picks_bans'][10]["hero_id"]
            ban4 = dpc.at[i, 'picks_bans'][11]["hero_id"]
            ban10 = dpc.at[i, 'picks_bans'][12]["hero_id"]
            ban5 = dpc.at[i, 'picks_bans'][13]["hero_id"]

            pick8 = dpc.at[i, 'picks_bans'][14]["hero_id"]
            pick3 = dpc.at[i, 'picks_bans'][15]["hero_id"]
            pick9 = dpc.at[i, 'picks_bans'][16]["hero_id"]
            pick4 = dpc.at[i, 'picks_bans'][17]["hero_id"]  

            ban11 = dpc.at[i, 'picks_bans'][18]["hero_id"]
            ban6 = dpc.at[i, 'picks_bans'][19]["hero_id"]

            pick5 = dpc.at[i, 'picks_bans'][20]["hero_id"]
            pick10 = dpc.at[i, 'picks_bans'][21]["hero_id"] 

            firstWin = dpc.at[i, 'radiant_win']
            secondWin = not dpc.at[i, 'radiant_win']

            firstPick = dpc.at[i, 'picks_bans'][0]["team"]

            if firstPick == 1:
                firstWin = not firstWin
                secondWin = not secondWin

            dpc_df = dpc_df.append({"pick_1": pick1, 
                                     "pick_2": pick2, 
                                     "pick_3": pick3, 
                                     "pick_4": pick4, 
                                     "pick_5": pick5,
                                     "ban_1": ban1, 
                                     "ban_2": ban2, 
                                     "ban_3": ban3, 
                                     "ban_4": ban4, 
                                     "ban_5": ban5,
                                     "ban_6": ban6,
                                     "win": firstWin}, ignore_index=True)
            dpc_df = dpc_df.append({"pick_1": pick6, 
                                     "pick_2": pick7, 
                                     "pick_3": pick8, 
                                     "pick_4": pick9, 
                                     "pick_5": pick10,
                                     "ban_1": ban6, 
                                     "ban_2": ban7, 
                                     "ban_3": ban8, 
                                     "ban_4": ban9, 
                                     "ban_5": ban10,
                                     "ban_6": ban11,
                                     "win": secondWin}, ignore_index=True)

dpc_df.head(10)

,pick_1,pick_2,pick_3,pick_4,pick_5,ban_1,ban_2,ban_3,ban_4,ban_5,ban_6,win
0,9,50,62,55,42,88,107,3,1,43,12,True
1,53,16,87,39,8,12,38,91,109,18,106,False
2,16,85,86,15,74,107,3,38,43,106,12,False
3,53,71,70,112,46,12,60,91,50,1,17,True
4,91,85,64,41,9,107,3,88,43,53,47,False
5,112,71,13,54,39,47,60,38,17,106,76,True
6,3,86,47,8,78,60,112,71,4,45,95,True
7,9,88,87,96,12,95,40,36,61,1,53,False
8,40,31,62,109,20,53,55,90,78,1,67,False
9,47,86,107,13,93,67,112,3,71,4,43,True


In [316]:
# testing on smaller subset of data

dpc_s = pd.read_json("data/DPC_small.json").transpose()
dpc_s.drop(["all_word_counts", "barracks_status_dire", "barracks_status_radiant", "chat", "cluster", 
          "cosmetics", "dire_score", "series_type", "skill", "teamfights", "throw", "tower_status_dire", 
          "tower_status_radiant", "loss", "leagueid", "duration", "engine", "first_blood_time", "radiant_xp_adv", 
          "replay_url", "radiant_gold_adv", "players", "positive_votes", "radiant_score", "negative_votes", 
          "objectives", "dire_team_id", "radiant_team_id", "human_players", "league", "lobby_type", 
          "draft_timings", "my_word_counts", "region", "replay_salt", "series_id"], axis=1, inplace=True)
dpc_s.drop([1], inplace=True)

In [318]:
dpc_s_df = pd.DataFrame(columns=["pick_1", "pick_2", "pick_3", "pick_4", "pick_5",
                                 "ban_1", "ban_2", "ban_3", "ban_4", "ban_5",
                                 "win"])

for i in dpc_s.index: 
    # print(dpc_s.at[i, 'patch'])
    ban1 = dpc_s.at[i, 'picks_bans'][0]["hero_id"]
    ban6 = dpc_s.at[i, 'picks_bans'][1]["hero_id"]
    ban2 = dpc_s.at[i, 'picks_bans'][2]["hero_id"]
    ban7 = dpc_s.at[i, 'picks_bans'][3]["hero_id"]
    # ban3
    # ban8
    pick1 = dpc_s.at[i, 'picks_bans'][4]["hero_id"]
    pick6 = dpc_s.at[i, 'picks_bans'][5]["hero_id"]
    pick7 = dpc_s.at[i, 'picks_bans'][6]["hero_id"]
    pick2 = dpc_s.at[i, 'picks_bans'][7]["hero_id"]
    
    ban8 = dpc_s.at[i, 'picks_bans'][8]["hero_id"]
    ban3 = dpc_s.at[i, 'picks_bans'][9]["hero_id"]
    ban9 = dpc_s.at[i, 'picks_bans'][10]["hero_id"]
    ban4 = dpc_s.at[i, 'picks_bans'][11]["hero_id"]
    
    pick8 = dpc_s.at[i, 'picks_bans'][12]["hero_id"]
    pick3 = dpc_s.at[i, 'picks_bans'][13]["hero_id"]
    pick9 = dpc_s.at[i, 'picks_bans'][14]["hero_id"]
    pick4 = dpc_s.at[i, 'picks_bans'][15]["hero_id"]  
    
    ban10 = dpc_s.at[i, 'picks_bans'][16]["hero_id"]
    ban5 = dpc_s.at[i, 'picks_bans'][17]["hero_id"]
    
    pick5 = dpc_s.at[i, 'picks_bans'][18]["hero_id"]
    pick10 = dpc_s.at[i, 'picks_bans'][19]["hero_id"] 
    
    firstWin = dpc_s.at[i, 'radiant_win']
    secondWin = not dpc_s.at[i, 'radiant_win']
    
    firstPick = dpc_s.at[i, 'picks_bans'][0]["team"]
    
    if firstPick == 1:
        firstWin = not firstWin
        secondWin = not secondWin
    
    dpc_s_df = dpc_s_df.append({"pick_1": pick1, 
                                 "pick_2": pick2, 
                                 "pick_3": pick3, 
                                 "pick_4": pick4, 
                                 "pick_5": pick5,
                                 "ban_1": ban1, 
                                 "ban_2": ban2, 
                                 "ban_3": ban3, 
                                 "ban_4": ban4, 
                                 "ban_5": ban5,
                                 "win": firstWin}, ignore_index=True)
    dpc_s_df = dpc_s_df.append({"pick_1": pick6, 
                                 "pick_2": pick7, 
                                 "pick_3": pick8, 
                                 "pick_4": pick9, 
                                 "pick_5": pick10,
                                 "ban_1": ban6, 
                                 "ban_2": ban7, 
                                 "ban_3": ban8, 
                                 "ban_4": ban9, 
                                 "ban_5": ban10,
                                 "win": secondWin}, ignore_index=True)

In [ ]:
# Does bans matter
# Does Team ID matter
# Only Picks
# based off patch